In [23]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display
import toml
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

def get_sheetdata(sheet_name):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load("/home/ciroshia/Downloads/github/comissao/.streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()

    df = pd.DataFrame(data)

    return df


def update_sheet(sheet_name,df):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load("/home/ciroshia/Downloads/github/comissao/.streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    worksheet.clear()
    set_with_dataframe(worksheet, df)

    return df


agendamento_path = "/home/ciroshia/Downloads/github/comissao/assets/agendamentos_colab-bi_2025-02-12 (1).xlsx"

agendamentos_df = pd.read_excel(agendamento_path)
procedimentos_df = get_sheetdata("procedimentos_padronizados")
prestadora_df = get_sheetdata("base_prestadoras")
comissao_df = get_sheetdata("comissoes")
tipo_prestadora_df = get_sheetdata("tipo_prestadora")

In [30]:
base_limpa = agendamentos_df.loc[agendamentos_df["Unidade do agendamento"] != 'plástica']
base_limpa = base_limpa.loc[base_limpa["Unidade do agendamento"] != 'homa']
base_limpa = pd.merge(base_limpa, prestadora_df, left_on="Prestador", right_on="nome_prestadora", how="left")
base_limpa = pd.merge(base_limpa, procedimentos_df, left_on="Procedimento", right_on="procedimento_crm", how="left")

# base_limpa = pd.merge(base_limpa, comissao_df, left_on=["procedimento_padronizado","tipo_prestadora"], right_on=["procedimento_padronizado","tipo_prestadora"], how="left")
# base_limpa = pd.merge(base_limpa, tipo_prestadora_df, left_on=["tipo_prestadora"], right_on=["tipo_prestadora"], how="left")

colunas = ['Procedimento', 'nome_prestadora','tipo_prestadora','procedimento_padronizado']

base_limpa = base_limpa.loc[~base_limpa["nome_prestadora"].isna(),colunas]
base_para_subir = base_limpa[["procedimento_padronizado","tipo_prestadora"]].drop_duplicates()
base_para_subir["comissão"] = 0
base_para_subir = base_para_subir.loc[~base_para_subir["procedimento_padronizado"].isna()]
base_para_subir.head(50)

,procedimento_padronizado,tipo_prestadora,comissão
1,PEELING DE DIAMANTE,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
4,DRENAGEM,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
5,RADIOFREQUÊNCIA,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
7,LIMPEZA DE PELE,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
10,CO2 ROSTO,PRESTAÇÃO DE SERVIÇOS - BIOMEDICA,0
43,PRÓ-LIPO,PRESTAÇÃO DE SERVIÇOS - BIOMEDICA,0
55,DRENAGEM (30 MIN),PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
62,SECAGEM,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
73,PRÓ-SHAPE,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
103,CARBOXITERAPIA,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0


In [31]:
update_sheet("comissoes",base_para_subir)

,procedimento_padronizado,tipo_prestadora,comissão
1,PEELING DE DIAMANTE,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
4,DRENAGEM,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
5,RADIOFREQUÊNCIA,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
7,LIMPEZA DE PELE,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
10,CO2 ROSTO,PRESTAÇÃO DE SERVIÇOS - BIOMEDICA,0
...,...,...,...
28105,PRÓ-MESO,PRESTAÇÃO DE SERVIÇOS - DENTISTA,0
32207,PRÓ-MESO,PRESTAÇÃO DE SERVIÇOS - SALÃO PARCEIRO,0
32394,PREENCHIMENTO,PRESTAÇÃO DE SERVIÇOS - MÉDICO DERMATO,0
37237,CO2 COLO,PRESTAÇÃO DE SERVIÇOS - BIOMEDICA,0
